In [27]:
from collections import OrderedDict
import numpy as np
import cv2
import dlib
import imutils
from PIL import Image
import face_recognition

def crop(image):
    image = face_recognition.load_image_file(image)
    face_locations = face_recognition.face_locations(image)
    for face_location in face_locations:
        top, right, bottom, left = face_location
        face_image = image[top:bottom, left:right]
        pil_image = Image.fromarray(face_image)
        pil_image.save('image/crop.png')

def avg(image):
    avg = cv2.mean(image)
    print(avg)
    rec = np.zeros((500, 500,3), np.uint8)
    rec = cv2.rectangle(rec, (0, 0), (500, 500), avg, -1)
    cv2.imshow('avg', rec)
    cv2.imwrite("image/skin_color.png", rec)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def cheek(image):
    CHEEK_IDXS = OrderedDict([("left_cheek", (2,4,31,40)),
                            ("right_cheek", (12,14,47,35))
                             ])
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    img = cv2.imread(image)
    img = imutils.resize(img, width=500)
    overlay = img.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    detections = detector(gray, 0)
    for k,d in enumerate(detections):
        shape = predictor(gray, d)
        for (_, name) in enumerate(CHEEK_IDXS.keys()):
            pts = np.zeros((len(CHEEK_IDXS[name]), 2), np.int32)
            for i,j in enumerate(CHEEK_IDXS[name]):
                pts[i] = [shape.part(j).x, shape.part(j).y]
            pts = pts.reshape((-1,1,2))
            cv2.polylines(overlay,[pts],True,(0,255,0),thickness = 2)
        aa = np.array(pts)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(overlay, [pts], True, (0, 255, 0), thickness=2)
        rect = cv2.boundingRect(aa)
        x, y, w, h = rect
        croped = img[y:y + h, x:x + w].copy()
        aa = aa - aa.min(axis=0)
        mask = np.zeros(croped.shape[:2], np.uint8)
        cv2.drawContours(mask, [aa], -1, (255, 255, 255), -1, cv2.LINE_AA)
        dst = cv2.bitwise_and(croped, croped, mask=mask)
        bg = np.ones_like(croped, np.uint8) * 255
        cv2.bitwise_not(bg, bg, mask=mask)
        dst2 = bg + dst
        dst2 = imutils.resize(dst2, width=600)
        cv2.imwrite("image/dst2.png", dst2)
        # cv2.imshow("Image1", dst)
        cv2.imshow("Image2", dst2)
        cv2.waitKey(0)
    cv2.destroyAllWindows()
    return dst


def center(image):
    im = Image.open(image)
    width = 500
    height = 500
    left = (width - 100)/2
    top = (height - 100)/2
    right = (width + 100)/2
    bottom = (height + 100)/2
    im = im.crop((left, top, right, bottom))
    im.show()
    im.save('image/center.png')



crop('image/obama.jpg')
aa = cheek('image/crop.png')
center('image/dst2.png')
im = cv2.imread('image/center.png')
avg(im)


(112.7972, 148.37460000000002, 207.6955, 0.0)
